<a href="https://colab.research.google.com/github/mheloisagalvao/cnn-ia/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown

In [9]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# Função para carregar e pré-processar imagens
def load_and_preprocess_images(folder, num_images_per_class=500):
    images = []
    labels = []
    class_indices = {}

    # Obtendo a lista de classes e atribuindo um índice a cada uma
    classes = os.listdir(folder)
    for i, fruit_class in enumerate(classes):
        class_indices[fruit_class] = i

    for fruit_class in classes:
        class_path = os.path.join(folder, fruit_class)
        class_images = []

        # Obtendo as imagens e labels da classe
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)

            try:
                img = cv2.imread(img_path)
                if img is not None and img.size > 0:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (224, 224))
                    class_images.append(img)
                    labels.append(class_indices[fruit_class])
            except Exception as e:
                print(f"Erro ao abrir {img_path}. O arquivo será ignorado.")

        # Adicionar as primeiras 'num_images_per_class' imagens da classe
        images.extend(class_images[:num_images_per_class])

    if not labels:
        print("Nenhuma label foi encontrada. Certifique-se de que as imagens estão sendo carregadas corretamente.")
        return None, None, None

    return np.array(images), to_categorical(labels), class_indices

# Carregando e pré-processando imagens
dataset_folder = 'fruits_dataset'
images, labels, class_indices = load_and_preprocess_images(dataset_folder, num_images_per_class=500)

if images is None or labels is None or class_indices is None:
    print("O carregamento e pré-processamento de imagens falhou.")
else:
    print("Shapes das imagens e labels:", images.shape, labels.shape)

    # Criando um ImageDataGenerator para pré-processamento e aumento de dados
    datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    # Criando geradores de dados para treino e validação
    train_generator = datagen.flow(
        x=images,
        y=labels,
        subset='training',
        batch_size=32,
        seed=42
    )

    validation_generator = datagen.flow(
        x=images,
        y=labels,
        subset='validation',
        batch_size=32,
        seed=42
    )

    # Certificando-se de que as labels estão em formato categórico
    categorical_labels = to_categorical(labels)

    # Definindo a arquitetura da CNN
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(class_indices), activation='softmax')
    ])

    # Compilando o modelo
    model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

    # Definindo checkpoints para salvar o melhor modelo
    checkpoint = ModelCheckpoint('model_best.h5', save_best_only=True)

    # Treinando o modelo
    model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[checkpoint])

    # Avaliando o modelo no conjunto de validação
    model.evaluate(validation_generator)

FileNotFoundError: ignored